In [ ]:
# import imutils
import cv2
import numpy as np 
import ipywidgets
class ShapeDetector:
    def __init__(self):
        pass
 
    def detect(self, c):
        # 初始化图片名称与大概的形状
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
         # 如果形状有3个顶点，则它是三角形
        if len(approx) == 3:
            shape = "triangle"

        # 如果形状有4个顶点，则它是正方形或矩形
        elif len(approx) == 4:
            
            # 计算轮廓的边界框，并使用
            # 计算纵横比的边界框
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)

            # 正方形的纵横比大约为
            # 等于1，否则，形状为矩形
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"

        # 否则，我们假设形状是圆形
        else:
            shape = "circle"

        # 返回形状的名称
        return shape
    
    # 找到最大轮廓
    def maxContour(self,cnts):
        area=[]
        if cnts :
            for k in range(len(cnts)):
                area.append(cv2.contourArea(cnts[k]))
            max_idx = np.argmax(np.array(area))
            return max_idx
    def openCamera(self,fps=30,width=120,heigh=80):
        #打开摄像头 
        cap=cv2.VideoCapture(0)
        if not cap.isOpened():
            cap=cv2.VideoCapture(1)
        if not cap.isOpened():
            raise IOError('Can not open video')
        # 设置摄像头参数
        if cap.isOpened():    
            cap.set(cv2.CAP_PROP_FPS, fps)
            cap.set(cv2.CAP_PROP_FRAME_WIDTH,width)
            cap.set(cv2.CAP_PROP_FRAME_HEIGHT,heigh)   
        wid = ipywidgets.Image()
        display(wid)
        return cap,wid
    # 显示输出图像
    def showImage(self,mask,image,wid):
        newblurred = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        # 拼接图像
        newimage = np.hstack((image,newblurred))
        wid.value = cv2.imencode('.jpg',newimage)[1].tobytes()
    
#形状检测模块
sd = ShapeDetector()

#打开摄像头 
cap,wid=sd.openCamera()

while True:
    ret,image = cap.read()    
    # 颜色空间转换
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

    #设置阈值
    lower_red = np.array([0, 43, 46])
    upper_red = np.array([10, 255, 255])
    # 根据侷值构建掩模
    mask0=cv2.inRange(hsv,lower_red,upper_red)
    #设置阈值
    lower_red = np.array([156, 43, 46])
    upper_red = np.array([180, 255, 255])
    # 根据侷值构建掩模
    mask1 = cv2.inRange(hsv,lower_red,upper_red)
    # 合并掩模
    mask = mask0 + mask1
    
    # 稍微模糊，
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)

    # 在阈值图像中查找轮廓并初始化
    # 形状检测器
    cnts,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # 找到最大轮廓
    max_idx = sd.maxContour(cnts)
    if max_idx: 
        c=cnts[max_idx]
        M = cv2.moments(c)
    
        # 计算轮廓的中心，然后检测
        #仅使用轮廓的形状
        cX = int((M["m10"] / M["m00"]))
        cY = int((M["m01"] / M["m00"]))
        shape = sd.detect(c)

        # 在图像上绘制轮廓和形状名称
        cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
        cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (255, 255, 255), 2)

    # 显示输出图像
    sd.showImage(mask,image,wid)

[ WARN:0@1.009] global /home/conda/feedstock_root/build_artifacts/libopencv_1644507584123/work/modules/videoio/src/cap_v4l.cpp (889) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


Image(value=b'')